In [68]:
# Loading Libraries
from datetime import datetime
from scipy import stats
from scipy.stats import norm, skew, kurtosis, jarque_bera
from statsmodels.tsa.stattools import acf, q_stat
from scipy.optimize import fsolve
import os
import seaborn as sns
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random 
import yfinance as yf

# ============================
# Initial Configuration
# ============================
# File paths and directories
img_directory = "Images_RA_Q3/"

# Create directory to save images (if it doesn't exist)
os.makedirs(img_directory, exist_ok=True)

# Set seed
random.seed(123)
np.random.seed(123)

Consider a bond with a 10-year maturity and annual coupon payments. This bond has a face value
of 100 and a coupon rate of 5%. Currently, the gross price of the bond is 99. Assume a year consists
of 360 days.
Compute the yield to maturity (YTM) for this bond, under the assumption that the daily fluctuations
in YTM follow independent and identically distributed (i.i.d.) Gaussian random variables with
a mean of 0 and a standard deviation of 0.006.

In [69]:
# Functions
def calc_yield(y,FV,P,C,T,X):
    return sum(C/(1+y)**(np.arange(1, T+1)-X/360)) + FV/(1+y)**(T-X/360) - P

def calc_bondP(y,FV,C,T,X):
    return sum(C/(1+y)**(np.arange(1, T+1)-X/360)) + FV/(1+y)**(T-X/360)

def calc_theta(y,FV,C,T):
    return sum(C*np.log(1+y)/((1+y)**(np.arange(2, T+2)))) + FV*np.log(1+y)/((1+y)**(T+1))

def calc_ddur(y,FV,C,T):
    return sum(-C*(np.arange(1, T+1))/((1+y)**(np.arange(2, T+2)))) - T*FV/((1+y)**(T+1))

def calc_conv(y,FV,C,T):
    return (1/2)*(sum((C*np.arange(1, T+1)*np.arange(2, T+2))/((1+y)**np.arange(3, T+3))) + T*(T+1)*FV/((1+y)**(T+2)))

YTM_orig = fsolve(calc_yield, args=(100,99,5,10,0), x0=0.01)
bp_orig = 99
print(YTM_orig,bp_orig)

[0.05130325] 99


In [70]:
#Check
print(calc_bondP(YTM_orig,100,5,10,0),calc_theta(YTM_orig,100,5,10)*1/360,calc_ddur(YTM_orig,100,5,10)*0.01, calc_conv(YTM_orig,100,5,10)*0.01*0.01)

[99.] [0.01308701] [-7.62469057] [0.36965634]


Estimate the probability of a 10% decline in the bond price within a 30-day period

In [71]:
# Price after 10% decline
P_10pd = 99*(1-0.1)

# Corresponding YTM as of 30th day
YTM_10pd = fsolve(calc_yield, args=(100,P_10pd,5,10,30), x0=0.01)

# Yield change
Chg = YTM_10pd - YTM_orig

# Gaussian percentile
1 - stats.norm.cdf(Chg, loc=0, scale=0.006*np.sqrt(30))

array([0.32862359])

Compute the Value at Risk (VaR) for your bond at a 99% confidence level across various horizons
(1, 10, 20, 30,...,90 days) using the following methods:
1. Exact formula (provide a detailed explanation of the procedure).
2. Exact formula via delta approximation (i.e., exploiting Taylor’s formula truncated to the
first order).
3. Exact formula via delta-gamma approximation (i.e., exploiting Taylor’s formula truncated
to the second order).
4. Monte Carlo simulation with at least 10,000 simulations and delta approximation.
5. Monte Carlo simulation with at least 10,000 simulations and delta-gamma approximation.
6. Monte Carlo simulation with at least 10,000 simulations and full revaluation.
7. Discuss and compare your results.

Monte Carlo simulations must be performed using at least 10,000 simulations. Greeks (i.e. Delta,
Gamma and Theta) can be computed analytically or via finite differences.

1.Exact formula (provide a detailed explanation of the procedure).

In [72]:
# Days dataframe
horizon = np.append([1], np.arange(10,100,10))
df = pd.DataFrame(index=horizon)

# Gaussian percentile
for i in horizon:
        tmp_yld_chg = stats.norm.ppf(0.99, loc=0, scale=0.006*np.sqrt(i))

# Yield change at 99th percentile
        tmp_yld_chg = stats.norm.ppf(0.99, loc=0, scale=0.006*np.sqrt(i))
        df.loc[i,"p99 y_chg"] = tmp_yld_chg
        df.loc[i,"p99 y_val"] = (YTM_orig + tmp_yld_chg)[0]

# No-yield change price         
        tmp_bp = calc_bondP(YTM_orig,100,5,10, i)[0]
        tmp_bp_chg = tmp_bp - bp_orig
        df.loc[i,"p_origYTM"] = tmp_bp        
        df.loc[i,"p_chg_origYTM"] = tmp_bp_chg
        
# Exact Formula VaR
        tmp_bp = calc_bondP(YTM_orig + tmp_yld_chg,100,5,10,i)[0]
        tmp_bp_chg = tmp_bp - bp_orig
        df.loc[i,"p99 p_exact"] = tmp_bp
        df.loc[i,"p99 p_chg_exact"] = tmp_bp_chg


2. Exact formula via delta approximation (i.e., exploiting Taylor’s formula truncated to the first order)

In [73]:
for i in horizon:
        tmp_yld_chg = df.loc[i,"p99 y_chg"]
        tmp_bp = bp_orig + calc_theta(YTM_orig,100,5,10)[0]*(i/360) + calc_ddur(YTM_orig,100,5,10)[0]*tmp_yld_chg
        tmp_bp_chg = tmp_bp - bp_orig
        df.loc[i,"p99 p_T1st"] = tmp_bp       
        df.loc[i,"p99 p_chg_T1st"] = tmp_bp_chg        


3.Exact formula via delta-gamma approximation (i.e., exploiting Taylor’s formula truncated to the second order).

In [74]:
# Gaussian percentile
for i in horizon:
        tmp_yld_chg = df.loc[i,"p99 y_chg"]
        tmp_bp = bp_orig + calc_theta(YTM_orig,100,5,10)[0]*(i/360) + calc_ddur(YTM_orig,100,5,10)[0]*tmp_yld_chg + calc_conv(YTM_orig,100,5,10)[0]*tmp_yld_chg**2
        tmp_bp_chg = tmp_bp - bp_orig
        df.loc[i,"p99 p_T2nd"] = tmp_bp
        df.loc[i,"p99 p_chg_T2nd"] = tmp_bp_chg
        

4. Monte Carlo simulation with at least 10,000 simulations and delta approximation.
5. Monte Carlo simulation with at least 10,000 simulations and delta-gamma approximation.
6. Monte Carlo simulation with at least 10,000 simulations and full revaluation

In [75]:
# 10,000 simulations for delta approx.
# should we separate the distribution for each case?
for i in horizon:
    
        tmp_yld_chg = stats.norm.rvs(loc=0, scale=0.006*np.sqrt(i), size=10000)
        tmp_bp_dt = np.zeros(len(tmp_yld_chg))
        tmp_bp_dtgm = np.zeros(len(tmp_yld_chg))
        tmp_bp_full = np.zeros(len(tmp_yld_chg))
        
        for j in range(len(tmp_yld_chg)):
            tmp_bp_dt[j] = bp_orig + calc_theta(YTM_orig,100,5,10)[0]*(i/360) + calc_ddur(YTM_orig,100,5,10)[0]*tmp_yld_chg[j]  
            tmp_bp_dtgm[j] = bp_orig + calc_theta(YTM_orig,100,5,10)[0]*(i/360) + calc_ddur(YTM_orig,100,5,10)[0]*tmp_yld_chg[j] + calc_conv(YTM_orig,100,5,10)[0]*tmp_yld_chg[j]**2 
            tmp_bp_full[j] = calc_bondP(YTM_orig+tmp_yld_chg[j],100,5,10,i)[0]
        
        tmp_yld_chg= np.sort(tmp_yld_chg)
        tmp_bp_dt = np.sort(tmp_bp_dt)[::-1]
        tmp_bp_dtgm = np.sort(tmp_bp_dtgm)[::-1]
        tmp_bp_full = np.sort(tmp_bp_full)[::-1]
        df.loc[i,"p99 y_chg_MC"] = tmp_yld_chg[int(len(tmp_yld_chg)*0.99) - 1]
        df.loc[i,"p99 p_MCdt"] = tmp_bp_dt[int(len(tmp_yld_chg)*0.99) - 1]
        df.loc[i,"p99 p_chg_MCdt"] = df.loc[i,"p99 p_MCdt"] - df.loc[i,"p_origYTM"]
        df.loc[i,"p99 p_MCdtgm"] = tmp_bp_dtgm[int(len(tmp_yld_chg)*0.99) - 1]
        df.loc[i,"p99 p_chg_MCdtgm"] = df.loc[i,"p99 p_MCdtgm"] - bp_orig
        df.loc[i,"p99 p_MCfull"] = tmp_bp_full[int(len(tmp_yld_chg)*0.99) - 1]
        df.loc[i,"p99 p_chg_MCfull"] = df.loc[i,"p99 p_MCfull"] - bp_orig
df

,p99 y_chg,p99 y_val,p_origYTM,p_chg_origYTM,p99 p_exact,p99 p_chg_exact,p99 p_T1st,p99 p_chg_T1st,p99 p_T2nd,p99 p_chg_T2nd,p99 y_chg_MC,p99 p_MCdt,p99 p_chg_MCdt,p99 p_MCdtgm,p99 p_chg_MCdtgm,p99 p_MCfull,p99 p_chg_MCfull
1,0.013958,0.065261,99.013759,0.013759,89.057924,-9.942076,88.370477,-10.629523,89.090672,-9.909328,0.013957,88.371621,-10.642139,89.091661,-9.908339,89.058923,-9.941077
10,0.044139,0.095443,99.137680,0.137680,71.703676,-27.296324,65.475983,-33.524017,72.677931,-26.322069,0.044921,64.880091,-34.257588,72.339332,-26.660668,71.312409,-27.687591
20,0.062422,0.113726,99.275551,0.275551,63.428028,-35.571972,51.666543,-47.333457,66.070439,-32.929561,0.061588,52.302439,-46.973112,66.324019,-32.675981,63.784361,-35.215639
30,0.076452,0.127755,99.413614,0.413614,58.005267,-40.994733,41.100637,-57.899363,62.706480,-36.293520,0.076811,40.826560,-58.587054,62.636053,-36.363947,57.870332,-41.129668
40,0.088279,0.139582,99.551869,0.551869,53.974623,-45.025377,32.213707,-66.786293,61.021498,-37.978502,0.089431,31.335251,-68.216618,61.130894,-37.869106,53.585597,-45.414403
50,0.098699,0.150002,99.690316,0.690316,50.788115,-48.211885,24.399736,-74.600264,60.409475,-38.590525,0.096100,26.380878,-73.309438,60.722289,-38.277711,51.603899,-47.396101
60,0.108119,0.159422,99.828956,0.828956,48.172932,-50.827068,17.347921,-81.652079,60.559608,-38.440392,0.106587,18.515868,-81.313088,60.621837,-38.378163,48.615383,-50.384617
70,0.116782,0.168085,99.967789,0.967789,45.971553,-53.028447,10.873630,-88.126370,61.287265,-37.712735,0.116068,11.418193,-88.549595,60.683126,-38.316874,46.162997,-52.837003
80,0.124845,0.176148,100.106814,1.106814,44.084127,-54.915873,4.856566,-94.143434,62.472149,-36.527851,0.124818,4.876898,-95.229916,60.804255,-38.195745,44.090804,-54.909196
90,0.132418,0.183721,100.246033,1.246033,42.443126,-56.556874,-0.786829,-99.786829,64.030702,-34.969298,0.128915,1.884460,-98.361574,60.907991,-38.092009,43.280517,-55.719483


In [82]:
# Take necessary outputs and modify the signs
df[['p99 p_chg_exact', 'p99 p_chg_T1st', 'p99 p_chg_T2nd', 'p99 p_chg_MCdt', 'p99 p_chg_MCdtgm', 'p99 p_chg_MCfull']] = -df[['p99 p_chg_exact', 'p99 p_chg_T1st', 'p99 p_chg_T2nd', 'p99 p_chg_MCdt', 'p99 p_chg_MCdtgm', 'p99 p_chg_MCfull']]
output = df[['p_origYTM','p_chg_origYTM','p99 y_chg','p99 y_chg_MC','p99 p_chg_exact', 'p99 p_chg_T1st', 'p99 p_chg_T2nd', 'p99 p_chg_MCdt', 'p99 p_chg_MCdtgm', 'p99 p_chg_MCfull']]
output.index.name = 'days'
output

,p_origYTM,p_chg_origYTM,p99 y_chg,p99 y_chg_MC,p99 p_chg_exact,p99 p_chg_T1st,p99 p_chg_T2nd,p99 p_chg_MCdt,p99 p_chg_MCdtgm,p99 p_chg_MCfull
days,,,,,,,,,,
1,99.013759,0.013759,0.013958,0.013957,9.942076,10.629523,9.909328,10.642139,9.908339,9.941077
10,99.137680,0.137680,0.044139,0.044921,27.296324,33.524017,26.322069,34.257588,26.660668,27.687591
20,99.275551,0.275551,0.062422,0.061588,35.571972,47.333457,32.929561,46.973112,32.675981,35.215639
30,99.413614,0.413614,0.076452,0.076811,40.994733,57.899363,36.293520,58.587054,36.363947,41.129668
40,99.551869,0.551869,0.088279,0.089431,45.025377,66.786293,37.978502,68.216618,37.869106,45.414403
50,99.690316,0.690316,0.098699,0.096100,48.211885,74.600264,38.590525,73.309438,38.277711,47.396101
60,99.828956,0.828956,0.108119,0.106587,50.827068,81.652079,38.440392,81.313088,38.378163,50.384617
70,99.967789,0.967789,0.116782,0.116068,53.028447,88.126370,37.712735,88.549595,38.316874,52.837003
80,100.106814,1.106814,0.124845,0.124818,54.915873,94.143434,36.527851,95.229916,38.195745,54.909196


Compute the Expected Shortfall of your bond at a 99% confidence level for different horizons
(1, 10, 20, 30,...,90 days) using your preferred method. Provide a detailed explanation of the
adopted procedure.

In [132]:
# ES: there's no analytical solution and taylor appriximation performs poorly in horizons over 20 days.
# Thus, I adopt here the MC simulation. (may be it is better to present with the confidence interval)
# we should be able to use the method in p.22 of 07.CoherentRiskMeasures

In [80]:
ES = pd.DataFrame(index=horizon)

for i in horizon:
    tmp_yld_chg = stats.norm.rvs(loc=0, scale=0.006*np.sqrt(i), size=10000)
    tmp_bp = np.zeros(len(tmp_yld_chg))
    tmp_bp_chg = np.zeros(len(tmp_yld_chg))
    n = int(len(tmp_yld_chg)*(1 - 0.99))
    
    for j in range(len(tmp_yld_chg)):
            tmp_bp[j] = calc_bondP(YTM_orig + tmp_yld_chg[j],100,5,10,i)[0]
            tmp_bp_chg[j] = tmp_bp[j] - bp_orig
            
    tmp_bp = np.sort(tmp_bp)[::-1]
    tmp_bp_chg = np.sort(tmp_bp_chg)[::-1]
    tmp_tail_p = tmp_bp[int(len(tmp_yld_chg)*0.99 - 1):int(len(tmp_yld_chg))].sum()
    tmp_tail_chg = tmp_bp_chg[int(len(tmp_yld_chg)*0.99 - 1):int(len(tmp_yld_chg))].sum()
    tmp_ES_p = tmp_tail_p/n #correction term will be zero by design
    tmp_ES_chg = tmp_tail_chg/n #correction term will be zero by design
    ES.loc[i,"p99 ES"] = (-tmp_ES_chg)
    ES.loc[i,"p99 p_chg_exact"] = -(df.loc[i,"p99 p_chg_exact"])
    ES.loc[i,"Difference"] = -(tmp_ES_chg - df.loc[i,"p99 p_chg_exact"])
    ES.loc[i,"p99 ES_price"] = -tmp_ES_p

ES

,p99 ES,p99 p_chg_exact,Difference,p99 ES_price
1,11.171915,9.942076,1.229839,-88.818085
10,31.359228,27.296324,4.062904,-68.630772
20,39.619732,35.571972,4.047760,-60.370268
30,45.719193,40.994733,4.724460,-54.270807
40,49.213718,45.025377,4.188341,-50.776282
50,51.885248,48.211885,3.673363,-48.104752
60,55.026950,50.827068,4.199882,-44.963050
70,57.662283,53.028447,4.633836,-42.327717
80,59.581218,54.915873,4.665344,-40.408782
90,62.380439,56.556874,5.823565,-37.609561


In [77]:
# Extract only necessary data
output = output.round(4)
output.to_latex('Q3_VaRtable_Latex.txt')

# Extract only necessary data
output = ES[['p99 ES', 'p99 p_chg_exact', 'Difference']]
output.index.name = 'days'
output = output.round(4)
output.to_latex('Q3_EStable_Latex.txt')


C:\Users\81902\AppData\Local\Temp\ipykernel_22292\408634750.py:5: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  output.to_latex('Q3_VaRtable_Latex.txt')
C:\Users\81902\AppData\Local\Temp\ipykernel_22292\408634750.py:11: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  output.to_latex('Q3_EStable_Latex.txt')
